In [ ]:
first_question_article = [
     {'article_id': '00000194-25d6-dcc4-a1d7-3df6703c0000',
      'question': 'אני רוצה לקרוא ביקורת על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז על הומו שחי במקסיקו בשנות ה-50 ומחפש גברים צעירים.'},
      {'article_id': '00000194-2618-dd68-a3be-e6fc06680000',
       'question': 'אני מחפש ביקורת על הסרט החדש עם ניקול קידמן, זה שהיא משחקת בו מנכ"לית חברת רובוטיקה שיש לה רומן עם מתמחה צעיר. הסרט הוקרן בפסטיבל ונציה, איך הוא?.'
       },
       {'article_id': '00000194-2681-ddb6-afdd-77e781220000',
        'question': 'אני מחפש ביקורת על הסרט החדש של עמוס גיתאי, זה שעוסק בשאלה למה יש מלחמות ומשתמש במכתבים של איינשטיין ופרויד.'
        },
        {'article_id': '00000194-2b97-d9c2-a79e-2bd7330c0000',
            'question': 'אפשר בבקשה ביקורת על הסרט החדש "נוספרטו" של רוברט אגרס? אני רוצה לדעת אם הוא נאמן למקורות של סרטי הערפדים הקלאסיים ואיך השחקנים הראשיים, במיוחד לילי-רוז דפ, משחקים?'
        },
        {'article_id': '00000194-319b-d555-abbc-b1dfdf250000',
            'question': '"אני מחפש ביקורת על סרט חדש שמוקרן בפסטיבל חיפה, על אישה אלכוהוליסטית בשם רונה שחוזרת לאי הולדתה בסקוטלנד כדי להתמודד עם ההתמכרות שלה. מישהי כתבה על זה ביקורת?'
        },
        {
            'article_id': '00000194-31db-ddaf-adb7-7bfbaf200000',
            'question': 'אני מחפש סרט איראני חדש שמתרחש בטהרן ועוסק במשפחה שאבא שלה שופט ורואים בו גם קטעים מההפגנות שם. ראיתי שהקרינו אותו בפסטיבל חיפה'
        },
        {'article_id': '00000194-34df-d39d-a196-b7ff86ce0000',
            'question': '"אני מחפש סדרת אנימציה חדשה בנטפליקס, משהו בסגנון באפי קוטלת הערפדים אבל עם מיתולוגיה סינית. שמעתי שיש סדרה על נערה סינית-אמריקאית שנלחמת בשדים, מישהו מכיר?',
        },
        {
            'article_id':"00000194-356b-de88-a3dc-75ef26460000",
            'question':"אוקיי, שמעתי על סרט חדש של שבי גביזון עם ריצ'רד גיר על אבא שמגלה שהיה לו בן שנהרג. איך קוראים לסרט ומה הוא מספר עליו?"
        },
        {
        'article_id':'00000194-4be5-dff1-a7bc-dbf57a210000',
        'question':'אני מחפש סרט על שני בני דודים יהודים אמריקאים שנוסעים לפולין בעקבות השואה. אחד מהם משחק בסדרה "יורשים". מישהו כתב וביים את הסרט והוא גם שחקן בסרט "הרשת החברתית".',
        },
        {
        'article_id':'00000194-6339-dabc-afff-6b7fb6e90000',
        'question':'אוקיי, שמעתי שיש סדרה חדשה בנטפליקס, מין מערבון כזה אבל ממש קשוח, עם הרבה אלימות וטבח. זה על איזה אישה שמנסה לשרוד עם הבן שלה בתקופה של מלחמת יוטה והטבח בהר מדוז. מישהו יודע על מה אני מדבר?',
        },
        {
        'article_id':'00000194-6e60-d9d3-a597-eeecd15d0000',
        'question':'"אני מחפש מידע על הסרט החדש של ארז תדמור, זה שמתרחש בשנות ה-50 ליד הכנרת ועוסק בפרטיזנים לשעבר שעובדים במפעל סודה. איך קוראים לסרט ומה העלילה שלו?',
        },
]

first_question_genre = [
    "אחי, תמליץ לי על איזה סרט אקשן טוב בנטפליקס? משהו חדש יחסית.",
    "מחפשת סדרה קלילה ומצחיקה, קומדיה כזאת, לא משנה לי באיזה שירות סטרימינג.",
    "מה כדאי לראות עכשיו ביס פלוס? בא לי איזה משהו מתח טוב.",
    "יש לך המלצה לסרט דרמה איכותי? עדיף משהו שזמין בהוט.",
    "אני והחבר'ה רוצים לראות סרט אימה הערב. יש משהו ממש מפחיד שאתה מכיר?",
    "תגיד, יש איזה סדרת מדע בדיוני שווה בפרטנר TV או בסלקום TV?",
    "בא לי לראות סרט ישראלי טוב, יש משהו מומלץ לאחרונה?",
    "מחפש סדרה לבינג', משהו שאפשר לטחון בסופ\"ש. ז'אנר פחות קריטי, העיקר שתהיה ממכרת.",
    "יש סרטי דוקו מעניינים בדיסני פלוס או באפל טי וי?",
    "סתם תן המלצה לסרט טוב שראית לאחרונה, לא משנה לי הז'אנר או הפלטפורמה."

]

genre_questions_as_dicts = [
    {'article_id': '', 'question': q} for q in first_question_genre
]

# איחוד שתי הרשימות לרשימה אחת
combined_questions = first_question_article + genre_questions_as_dicts

second_question = 'איזה יצירה המבקר העדיף יותר?'
thierd_question = 'אם הייתי אומר לך שאני במיוחד רגיש לסיומים מאכזבים/לא סגורים, איזו יצרה עדיפה מבין אלו שהמלצת?'



In [ ]:
# Follow-up questions appropriate after a *general* first question that returned two recommendations
general_follow_up_questions = [
    "תסביר בפירוט איך הגעת להמלצה הזאת, ואיזה קטעים ספציפיים מתוך הכתבות תמכו בבחירה.",
    "יש לי רתיעה מאלימות גרפית. האם הסרט שהמלצת עליו כולל סצנות כאלה? תצטט מהמקורות שלך.",
    "האם יש סרט מתח ישראלי שעונה על אותם קריטריונים? נמק עם מקורות.",
    "אני מעדיף סרטים קצרים (עד 100 דקות). תעדכן את ההמלצה בהתאם.",
    "הסרט שהצעת כבר ראיתי. הבא המלצה אחרת, אבל שמור על הסיבות לבחירה שהצגת.",
    "תן לי טריילר רשמי (YouTube) וקישור לביקורת מהשנה האחרונה.",
    "אפשר להמליץ לי על סדרת טלוויזיה באותו סגנון, לפי אותם מקורות?",
    "מי קהל היעד העיקרי של הסרט לפי הנתונים במקורות?",
    "שכנע צופה שלא אוהב סרטי מתח מדוע כדאי לו בכל זאת לראות את הסרט שהצעת.",
    "האם קיימות אזהרות תוכן (trigger warnings) רלוונטיות? הבא ציטוט מפורט."
]

# Follow-up questions appropriate after a *specific* first question that returned a single recommendation
specific_follow_up_questions = [
    "תוכל להשוות בין הסרט שהצעת לבין ‘Seven’ מבחינת עלילה וטון?",
    "האם בסרט שהמלצת יש ייצוג נשי משמעותי? צטט ביקורות שמדברות על כך.",
    "מהן שלוש נקודות הביקורת העיקריות שהוזכרו נגד הסרט במקורות שהבאת?",
    "אשמח לתקציר נטול ספוילרים של 100 מילים בדיוק.",
    "מי מהשחקנים הראשיים היה מועמד/זכה בפרסים, ועל אילו תפקידים אחרים הם ידועים?",
    "האם הסרט מבוסס על ספר? אם כן, מה ההבדלים העיקריים לפי הכתבות?",
    "מהו המוטיב המרכזי (theme) ששב ועולה בביקורות, ואיך הוא בא לידי ביטוי בסצנה ספציפית?",
    "האם יש פסקול בולט במיוחד? נסח ביקורת קצרה על המוזיקה וציין ציטוט ממקור.",
    "מהן הנקודות שהמבקרים הכי שיבחו בצילום (cinematography)? הבא שתי דוגמאות וציטוטים.",
    "מבחינת Diversity & Inclusion—האם הסרט קיבל ביקורת חיובית או שלילית? הבא נתונים.",
    "האם יש Easter Eggs נסתרים שהמבקרים הזכירו? פרט שניים עם הפניות."
]

second_question_article = specific_follow_up_questions + general_follow_up_questions

In [ ]:
len(first_question_article)

In [ ]:
from src.llm_api_client import LLMClient
from config.load_config import load_config

from test_antrpoic.antropic import ChatBot
from src.redis_chat_history import RedisChatHistory
import streamlit as st


prompts = load_config("config/prompts.yaml")
sys_instruct = prompts.get("system_instructions")
config = load_config("config/config.yaml")
llm_cfg = config.get("llm", {})

redis_store = RedisChatHistory()
llm_client = LLMClient(
    model_name=llm_cfg.get("llm_model_name"),
    api_key=llm_cfg.get("GOOGLE_API_KEY"),
    sys_instruct=sys_instruct,
    config=config,
    redis_store=redis_store,
)
# Initialize the LLM client antropic
antrp = ChatBot(config)

result = []

import numpy as np
counter = np.random.randint(0, 9999999999)


for q in combined_questions:
    st.session_state.messages = []
    counter += 1


    print('\n')
    print("="*20)
    article_id = q.get('article_id')
    question = q.get('question')
    print(f"Article ID: {article_id}")
    print(f"Question: {question}")
    
    full_response = ""
    ans = llm_client.streaming_message(question,f'{counter}')
    async for chunk in ans:
        print(chunk, end='', flush=True)
        if chunk:
            full_response += chunk


    second_question = second_question_article.pop(0)

    print('\n')
    print("="*20)
    print(f"Second Question: {second_question}")
    second_answer = ""
    ans = llm_client.streaming_message(second_question,f'{counter}')
    async for chunk in ans:
        if chunk:
            second_answer += chunk
        print(chunk, end='', flush=True)

    # thierd_answer = ""
    # ans = llm_client.streaming_message(thierd_question,f'{counter}')
    # async for chunk in ans:
    #     if chunk:
    #         thierd_answer += chunk
    #     print(chunk, end='', flush=True)



    # print('antrpoic answer')
    # antropic_answer, state = antrp.process_user_input(question,st.session_state)
    # print(antropic_answer)
    # second_answer_antrpoic, state = antrp.process_user_input(second_question,state)
    # print(second_answer_antrpoic)
    # thierd_answer_antrpoic, state = antrp.process_user_input(thierd_question,state)
    # print(thierd_answer_antrpoic)

    result.append({
        'article_id': article_id,
        'question': question,
        'answer_gemini': full_response,
        'second_question': second_question,
        'second_answer_gemini': second_answer,
        # 'answer_antrp': antropic_answer,
        # 'second_answer_gemini': second_answer,
        # 'second_answer_antrp': second_answer_antrpoic,
        # 'thierd_answer_gemini': thierd_answer,
        # 'thierd_answer_antrp': thierd_answer_antrpoic
    })
    
    
    


נסה לחזר אחרי סוגת המותחן הארוטי ומציג דינמיקה מסוכנת בין הבוסית לעובד הזוטר. המבקר מציין שהיבט המתח בסרט דל, והוא מתמקד יותר בפתולוגיה של הדמות הראשית. הביקורת מציינת שהשחקנים טובים, במיוחד ניקול קידמן, אנטוניו בנדרס שמגלם את בעלה, והאריס דיקינסון בתפקיד המתמחה. למרות השחקנים הטובים, המבקר פחות התלהב מהסרט עצמו וטוען שהוא מציע צפייה שמגרה בעיקר את המוח ולאו דווקא מעורר חשקים או ריגוש אמיתי. הוא מסכם שהסרט הוא בקושי סרט קולנוע ומביע אכזבה.

מעניין לשמוע את דעתך אחרי שתצפה בו! תכננת ללכת לראות אותו בקולנוע?{"version": "1.0", "model": "gemini-2.5-flash-preview-04-17", "user_id": "5206454991", "input_tokens": 4871, "output_tokens": 358, "rag_speed": 0, "llm_speed": 0}

{"asctime": "2025-05-05 13:00:28,290", "name": "UNKNOWN_APP_NAME_logger", "levelname": "DEBUG", "message": "Loaded history for user 5206454991 from Redis."}




Second Question: האם בסרט שהמלצת יש ייצוג נשי משמעותי? צטט ביקורות שמדברות על כך.
שאלה מצוינת! בדקתי שוב את הביקורת על "בייביגירל" כדי לראות מה נאמר בה על ייצוג נשי מעבר לדמותה הראשית של רומי בכיכובה של ניקול קידמן.

הביקורת מציינת שישנן דמויות נשיות נוספות בסרט, אך מתייחסת אליהן בצורה די ביקורתית מבחינת העומק שלהן. הנה ציטוט מהביקורת בנושא:

"יש עוד נשים בסרט, למשל בנותיהם של הזוג המרכזי או העוזרת השאפתנית של רומי. מגלמת אותה סופי ויילד ("נגעת נרצחת") וכמו הרבה דמויות משנה היא יותר ייצוג רעיוני של ויכוחים אינטרנטיים מאשר אישה של ממש — דוגלת בקידום נשים אבל מתגלה כאינטרסנטית."

לפי המבקר, נראה שהדמויות הנשיות המשניות מרגישות פחות כמו נשים אמיתיות ויותר כמו נקודות מבט המייצגות רעיונות או דיונים עכשוויים.

האם יש לך ז'אנרים או סוגי סרטים מסוימים עם ייצוג נשי חזק שמעניינים אותך? אשמח לחפש המלצות נוספות!{"version": "1.0", "model": "gemini-2.5-flash-preview-04-17", "user_id": "5206454991", "input_tokens": 490, "output_tokens": 330, "rag_speed": 0, "llm_speed": 0}

2025-05-05 13:00:37.336 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 13:00:37.337 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
{"asctime": "2025-05-05 13:00:37,420", "name": "UNKNOWN_APP_NAME_logger", "levelname": "DEBUG", "message": "No history found for user 5206454992 in Redis."}




Article ID: 00000194-2681-ddb6-afdd-77e781220000
Question: אני מחפש ביקורת על הסרט החדש של עמוס גיתאי, זה שעוסק בשאלה למה יש מלחמות ומשתמש במכתבים של איינשטיין ופרויד.


...



Second Question: מהן שלוש נקודות הביקורת העיקריות שהוזכרו נגד הסרט במקורות שהבאת?
על פי הביקורת שמצאתי, שלוש נקודות הביקורת העיקריות נגד הסרט "למה מלחמה" הן:

1.  **הישענות רבה מדי על מלל ומונולוגים:** רוב הסרט מורכב ממונולוגים ארוכים וסצינות עם משפטים שאינם מאוד מעמיקים ולעתים חוזרים על עצמם, מה שהופך אותו לנדוש ומיושן בחלקיו.
2.  **ביצוע מרושל או עצל בחלקים:** למרות רעיונות ויזואליים מעניינים, ישנם דימויים או אלמנטים כמו התחפושת של איינשטיין שנראים מרושלים או מבוצעים בעצלות, מה שפוגע ברצינות הסרט ויוצר אפקט לא מכוון.
3.  **תחושת טרחנות וחוסר גיבוש:** הסרט נתפס כהרהור לא מגובש או חד, ולמרות שהוא מעלה שאלות רבות בהקשרים שונים, התחושה המתקבלת היא של "דיבור טרחני אינסופי" במקום תוצאה עשירה.

האם תרצה/י לשמוע על סרטים אחרים של עמוס גיתאי או אולי לקבל המלצות לסרטים העוסקים בנושאים דומים?{"version": "1.0", "model": "gemini-2.5-flash-preview-04-17", "user_id": "5206454992", "input_tokens": 549, "output_tokens": 329, "rag_speed": 0, "llm_speed": 0}

2025-05-05 13:00:51.031 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-05 13:00:51.031 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
{"asctime": "2025-05-05 13:00:51,117", "name": "UNKNOWN_APP_NAME_logger", "levelname": "DEBUG", "message": "No history found for user 5206454993 in Redis."}




Article ID: 00000194-2b97-d9c2-a79e-2bd7330c0000
Question: אפשר בבקשה ביקורת על הסרט החדש "נוספרטו" של רוברט אגרס? אני רוצה לדעת אם הוא נאמן למקורות של סרטי הערפדים הקלאסיים ואיך השחקנים הראשיים, במיוחד לילי-רוז דפ, משחקים?


{"asctime": "2025-05-05 13:00:52,831", "name": "UNKNOWN_APP_NAME_logger", "levelname": "INFO", "message": "query=נוספרטו רוברט אגרס, streaming=[], genres=[], media_type=None"}
{"asctime": "2025-05-05 13:00:53,524", "name": "UNKNOWN_APP_NAME_logger", "levelname": "DEBUG", "message": "Raw search_results for cost tracking: [{\"actors\": [\"לילי־רוז דפ\", \"ביל סקארסגארד\", \"ניקולס הולט\", \"סיימון מקברני\", \"אהרון טיילור־ג'ונסון\", \"אמה קורין\", \"וילם דפו\", \"ראלף אינסון\"], \"article_id\": \"00000194-2b97-d9c2-a79e-2bd7330c0000\", \"article_name\": \"\\\"נוספרטו\\\" משיב את הכבוד לסרטי הערפדים\", \"article_type\": \"ReviewArticle\", \"author_image_square\": \"https://img.haarets.co.il/bs/00000184-dc9f-d3c9-a9a5-fd9f3f7f0001/8e/1e/784d834a44a792ba9fc12fbe9fdc/oronshamir2022.jpg?precrop=1661,1661,x90,y0\", \"brand\": \"HTZ\", \"create_by_AI\": false, \"director\": [\"רוברט אגרס\"], \"distribution_platform\": [\"בתי קולנוע\"], \"genre\": [\"אימה\"], \"image_vertical\": \"https://img.ha

[{"article_id": "00000194-2b97-d9c2-a79e-2bd7330c0000", "article_name": "\"נוספרטו\" משיב את הכבוד לסרטי הערפדים", "writer_name": ["אורון שמיר"], "review_type": "Movie", "genre": ["אימה"], "distribution_platform": ["בתי קולנוע"], "short_summary": "אגרס משיב כבוד לסרטי ערפדים, יצירת אימה גותית מצמררת.", "summary": "רוברט אגרס משיב את הכבוד לסרטי הערפדים עם 'נוספרטו' המצמרר. יצירה גותית אפלה ומהפנטת, מחווה לקלאסיקה אך מקורית, עם הופעה אייקונית של סקארסגארד. חוויה ויזואלית שמהלכת אימה.", "image_vertical": "https://img.haarets.co.il/bs/00000194-2b97-d9c2-a79e-2bd732250000/a7/fd/25216abf4c41a4255c64b223ee11/59940541.JPG?precrop=818,963,x315,y0", "author_image_square": "https://img.haarets.co.il/bs/00000184-dc9f-d3c9-a9a5-fd9f3f7f0001/8e/1e/784d834a44a792ba9fc12fbe9fdc/oronshamir2022.jpg?precrop=1661,1661,x90,y0", "url": "https://www.haaretz.co.il/gallery/cinema/movie-reviews/2025-01-03/ty-article-review/.premium/00000194-2b97-d9c2-a79e-2bd7330c0000"}, {"article_id": "00000195-d886-d1f1-a7d5

In [ ]:
import pandas as pd
pd.DataFrame(result).to_excel('first_question_article.xlsx', index=False)